In [1]:
import pandas as pd
import math

In [2]:
from datetime import datetime
date = datetime.fromtimestamp(1584148400	).date()
print(date)

2020-03-13


In [17]:
main = pd.read_excel("statePop.xlsx")
AL_data = pd.read_excel("States_data/AL_data.xlsx")

In [35]:
R_0 = 2                                # Basic reproduction ratio
incubationDur = 6.4                    # Average incubation per
infectionDur = 7                       # Average dur of infection
dt = 1                                 # Time step
peopleSeen = 10                        # Avg number of people seen by average American

alpha = 1 - math.exp(-1/incubationDur) # Daily prob of individual becoming infectious
gamma = 1 - math.exp(-1/infectionDur)  # Daily prob an individual recovers
beta = R_0 * gamma                     # Scaled transmission rate

In [39]:
def numSus(t, pop):                          # Susceptible to Exposed func
    infProb = I[t] * peopleSeen * pop/S[i]
    return infProb * dt                      # How many people infected people have met with, weighted by population outdoors

def numInfected(t):
    if (t < 28):
        asymp = .4                           # (1-asymp) is prob of asympotomatic
    else: 
        asymp = .8
    return (1-asymp) * alpha * E[t] * dt     # daily prob of being infected after exposure * exposed pop

def numRecovery(t):
    return gamma * I[t] * dt                 # daily prob that an individual recovers * number of infected people

In [40]:
def sGrowth(t, pop): # Current pop (S[t]) - exposed
    return S[t] - numSus(t, pop)

def eGrowth(t, pop): # number of people who have been exposed (numSus) - number of people infected in last timestep
    return numSus(t, pop) - numInfected(t)

def iGrowth(t): # Infected previously + newly infected - recovered + growth in actual infected cases
    return I[t] + numInfected(t) - numRecovery(t) 

def rGrowth(t): # Recovered
    return R[t] + numRecovery(t)

In [42]:
num = 24 + 21
S = [main['Population'].iloc[0]]
E = [0]
I = [0]
R = [0]
timeIndex = 0
firstCase = False

for i in range(num):
    if (i < 2):
        datePath = '3-0' + str(i+8)
    elif (i >= 2 and i < 24):
        datePath = '3-' + str(i+8)
    elif (i >= 24 and i < 33):
        datePath = '4-0' + str(i-23)
    else:
        datePath = '4-' + str(i-23)
    dateNow = datePath + '-2020'
    ourDate = datetime.strptime(dateNow, '%m-%d-%Y').date()
    
    if (True):
        infectedNum = 0
        for j in range(timeIndex, AL_data.shape[0]):
            covidDate = AL_data['seconds_since_Epoch'].iloc[j]
            covidDate = datetime.fromtimestamp(covidDate).date()
            if (ourDate == covidDate):
                infectedNum = int(AL_data.iloc[j][2])
                if (infectedNum != 0):
                    firstCase = True
            elif (covidDate > ourDate):
                timeIndex = j
                break
        print(infectedNum)
        if (infectedNum == 6):
            I[i] += infectedNum
    
    path = "data/data" + datePath + '.xlsx'
    data = pd.read_excel(path)
    percentAtHome = data[data['stname'] == 'Alabama']['percentAtHome'][0]
    atHome = .1 * percentAtHome/100 * S[i]
    
    S.append(sGrowth(i, S[i] - atHome))
    E.append(eGrowth(i, S[i] - atHome))
    I.append(iGrowth(i))
    R.append(rGrowth(i))
    print("Iteration: " + ourDate.strftime("%m/%d/%Y"))
    print("Susceptible: ", end="")
    print(S)
    print("Exposed: ", end="")
    print(E)
    print("Infected: ", end="")
    print(I)
    print("Recovered: ", end="")
    print(R)
    print("\n")

0
Iteration: 03/08/2020
Susceptible: [4903185, 4903185.0]
Exposed: [0, 0.0]
Infected: [0, 0.0]
Recovered: [0, 0.0]


0
Iteration: 03/09/2020
Susceptible: [4903185, 4903185.0, 4903185.0]
Exposed: [0, 0.0, 0.0]
Infected: [0, 0.0, 0.0]
Recovered: [0, 0.0, 0.0]


0
Iteration: 03/10/2020
Susceptible: [4903185, 4903185.0, 4903185.0, 4903185.0]
Exposed: [0, 0.0, 0.0, 0.0]
Infected: [0, 0.0, 0.0, 0.0]
Recovered: [0, 0.0, 0.0, 0.0]


0
Iteration: 03/11/2020
Susceptible: [4903185, 4903185.0, 4903185.0, 4903185.0, 4903185.0]
Exposed: [0, 0.0, 0.0, 0.0, 0.0]
Infected: [0, 0.0, 0.0, 0.0, 0.0]
Recovered: [0, 0.0, 0.0, 0.0, 0.0]


0
Iteration: 03/12/2020
Susceptible: [4903185, 4903185.0, 4903185.0, 4903185.0, 4903185.0, 4903185.0]
Exposed: [0, 0.0, 0.0, 0.0, 0.0, 0.0]
Infected: [0, 0.0, 0.0, 0.0, 0.0, 0.0]
Recovered: [0, 0.0, 0.0, 0.0, 0.0, 0.0]


6
Iteration: 03/13/2020
Susceptible: [4903185, 4903185.0, 4903185.0, 4903185.0, 4903185.0, 4903185.0, 4903125.8509552805]
Exposed: [0, 0.0, 0.0, 0.0, 0.0, 

Iteration: 03/27/2020
Susceptible: [4903185, 4903185.0, 4903185.0, 4903185.0, 4903185.0, 4903185.0, 4903125.8509552805, 4903074.906833923, 4902980.951517782, 4902860.192356975, 4902679.381508737, 4902426.944046464, 4902063.431771694, 4901548.3085774835, 4900818.076465948, 4899778.174503995, 4898293.430504834, 4896177.040579659, 4893152.411492201, 4888843.525076144, 4882701.616488395]
Exposed: [0, 0.0, 0.0, 0.0, 0.0, 0.0, 59.149044719810355, 45.81040993420866, 89.97930222802394, 112.94960489934056, 171.0076353610024, 237.59523016068965, 342.89071861897594, 485.36274740768414, 688.1061179181206, 980.1793027933792, 1399.6714894253478, 1994.9085124669116, 2851.4853847049094, 4061.398005049119, 5789.408468291857]
Infected: [0, 0.0, 0.0, 0.0, 0.0, 6.0, 5.2012673985010895, 9.642575180865393, 12.334949233939271, 18.502450793183172, 25.842578560307743, 37.24459233913511, 52.90807013532845, 75.62528352143923, 107.68388056513447, 153.07143538037667, 217.7667541496556, 310.2585991806412, 442.10002

Exposed: [0, 0.0, 0.0, 0.0, 0.0, 0.0, 59.149044719810355, 45.81040993420866, 89.97930222802394, 112.94960489934056, 171.0076353610024, 237.59523016068965, 342.89071861897594, 485.36274740768414, 688.1061179181206, 980.1793027933792, 1399.6714894253478, 1994.9085124669116, 2851.4853847049094, 4061.398005049119, 5789.408468291857, 8222.766697197709, 11664.492581438248, 16683.618362595087, 23656.5399299424, 33748.462359210724, 48015.21248145881, 68634.04882936516, 97331.65245727128, 142849.06913113044, 150266.5848202989, 169237.52569623763, 187648.2341733574, 209496.73354247888, 233286.6755126, 259448.4641449658, 283793.4519788416, 321096.449020767, 355592.1700477746, 396770.9859167101]
Infected: [0, 0.0, 0.0, 0.0, 0.0, 6.0, 5.2012673985010895, 9.642575180865393, 12.334949233939271, 18.502450793183172, 25.842578560307743, 37.24459233913511, 52.90807013532845, 75.62528352143923, 107.68388056513447, 153.07143538037667, 217.7667541496556, 310.2585991806412, 442.1000255906703, 630.73515267083